# Effect of Account Objective Completion

- Since 2022/09/03, users sponsored by Fairwind Credit Union got access to 3 bonus one-time objectives (Complete 5 Modules, Complete 10 Modules, Complete 15 Modules)
- This exploration aims to answer whether the one-time bonus objectives are working.

#### Import packages

In [1]:
import os
import pandas as pd
import mysql.connector
import plotly.express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls

#chart_studio credentials
chart_studio.tools.set_credentials_file(username = os.environ.get('cs_user'), api_key = os.environ.get('cs_key'))

In [2]:
#Pandas number of rows displayed (Set None for all rows)
#pd.set_option('display.max_rows', 10)

#### SQL Setup

In [2]:
usr = os.environ.get('sql_user')
pwd = os.environ.get('sql_pwd')
host = os.environ.get('sql_host')
db = os.environ.get('sql_db')

def query(sql):
    
    cnx = mysql.connector.connect(user=usr, 
                              password=pwd,
                              host=host,
                              database=db)
    
    df = pd.read_sql(sql, cnx)
    cnx.close()
    return df

## Fairwind Users

In [4]:
#Every objective claimed by Fairwind user:

fairwind_all_obj_query = '''
SELECT
	user_id,
    date_claimed,
    o.time_period,
    o.name
    FROM zogo_production.objective_claim oc
LEFT JOIN user u ON u.id = oc.user_id
LEFT JOIN objective o ON o.id = oc.objective_id
WHERE u.institution_id = 20048
'''

fairwind_all_obj = query(fairwind_all_obj_query)

fairwind_all_obj


,user_id,date_claimed,time_period,name
0,1406193,2022-07-07 14:18:48,day,Daily Login
1,1406193,2022-07-07 16:10:31,day,Join Daily Trivia
2,1406193,2022-07-07 16:13:03,day,Complete 3 Modules
3,1406972,2022-07-07 17:38:22,day,Daily Login
4,1406193,2022-07-20 19:01:09,day,Daily Login
...,...,...,...,...
9434,2171675,2022-10-20 19:21:58,week,Weekly Login
9435,2171675,2022-10-20 19:24:24,day,Complete 3 Modules
9436,1963228,2022-10-20 19:45:32,day,Daily Login
9437,1963228,2022-10-20 19:47:23,day,Join Daily Trivia


In [5]:
#Every one-time objective claimed by Fairwind users:

fairwind_onetime_obj_query = '''
SELECT
	user_id,
    date_claimed,
    o.time_period,
    o.name
    FROM zogo_production.objective_claim oc
LEFT JOIN user u ON u.id = oc.user_id
LEFT JOIN objective o ON o.id = oc.objective_id
WHERE u.institution_id = 20048
    AND o.time_period = 'one_time'
'''

fairwind_onetime_obj = query(fairwind_onetime_obj_query)

fairwind_onetime_obj

,user_id,date_claimed,time_period,name
0,1183866,2022-09-03 00:03:05,one_time,Complete 5 Modules
1,1976016,2022-09-03 00:06:21,one_time,Complete 5 Modules
2,1967467,2022-09-03 00:24:22,one_time,Complete 5 Modules
3,1964181,2022-09-03 00:33:50,one_time,Complete 5 Modules
4,1967076,2022-09-03 01:00:04,one_time,Complete 5 Modules
...,...,...,...,...
556,2169860,2022-10-13 23:18:43,one_time,Complete 25 Modules
557,2175129,2022-10-15 22:21:41,one_time,Complete 25 Modules
558,2176853,2022-10-16 12:48:00,one_time,Complete 25 Modules
559,2019402,2022-10-17 14:10:13,one_time,Complete 25 Modules


In [6]:
#Number of one-time objectives completed by Fairwind users who completed at least one one-time objective

fairwind_onetime_users_query = '''
SELECT
	user_id,
    COUNT(o.name) one_time_obj_count
    FROM zogo_production.objective_claim oc
LEFT JOIN user u ON u.id = oc.user_id
LEFT JOIN objective o ON o.id = oc.objective_id
WHERE u.institution_id = 20048
    AND o.time_period = 'one_time'
GROUP BY user_id
'''

fairwind_onetime_users = query(fairwind_onetime_users_query)

fairwind_onetime_users

,user_id,one_time_obj_count
0,25604,3
1,136491,2
2,136814,2
3,137970,2
4,543897,3
...,...,...
224,2176853,3
225,2181341,3
226,2181780,1
227,2184055,2


In [7]:
#Users who have logged in after rollout of one-time objective

fairwind_every_user_query = '''
SELECT 
	DISTINCT(lt.user_id),
    min(lt.date_created) AS date_login
 FROM zogo_production.login_tracker lt
LEFT JOIN user u ON u.id = lt.user_id
WHERE u.institution_id = 20048
        AND lt.date_created >= '20220903'
GROUP BY user_id
'''

fairwind_every_user = query(fairwind_every_user_query)

fairwind_every_user

,user_id,date_login
0,25604,2022-09-03 21:41:00
1,136491,2022-10-05 17:58:23
2,136814,2022-10-12 10:47:13
3,137970,2022-09-22 18:56:06
4,358789,2022-09-18 23:58:15
...,...,...
4197,2188014,2022-10-20 17:31:55
4198,2188027,2022-10-20 17:40:29
4199,2188184,2022-10-20 19:18:24
4200,2188199,2022-10-20 19:22:21


#### Modules/user based on cohort

In [8]:
#(Number of modules completed)/(user) for all users:

mod_per_user_all_query = '''
SELECT
    (SELECT count(*)
    FROM education_completed_module ecm
    JOIN user u on u.id = ecm.user_id
    WHERE u.institution_id = 20048
        AND ecm.date_completed >= '20220903'
        AND u.id IN 
			(
            SELECT 
				DISTINCT(lt.user_id)
			FROM zogo_production.login_tracker lt
			LEFT JOIN user u ON u.id = lt.user_id
			WHERE u.institution_id = 20048
			AND lt.date_created >= '20220903'
			GROUP BY user_id
            )
	)
/
    (SELECT COUNT(*)
    FROM (SELECT 
				DISTINCT(lt.user_id),
				min(lt.date_created) AS date_login
			FROM zogo_production.login_tracker lt
			LEFT JOIN user u ON u.id = lt.user_id
			WHERE u.institution_id = 20048
			AND lt.date_created >= '20220903'
			GROUP BY user_id) all_users
	) AS module_per_user
'''

query(mod_per_user_all_query)

,module_per_user
0,12.3129


In [9]:
#(Number of modules completed)/(user) for users who claimed at least one one-time objective 

mod_per_user_one_obj_query = '''
SELECT
    (SELECT count(*)
    FROM education_completed_module ecm
    JOIN user u on u.id = ecm.user_id
    WHERE u.institution_id = 20048
        AND ecm.date_completed >= '20220903'
        AND u.id IN 
			(
            SELECT user_id
				FROM
				(SELECT
					user_id,
					COUNT(o.name) one_time_obj_count
				FROM zogo_production.objective_claim oc
				LEFT JOIN user u ON u.id = oc.user_id
				LEFT JOIN objective o ON o.id = oc.objective_id
				WHERE u.institution_id = 20048
					AND o.time_period = 'one_time'
				GROUP BY user_id
				) one_time_user
				WHERE one_time_obj_count >= 1
            )
	)
/
    (SELECT count(user_id)
		FROM
		(SELECT
			user_id,
			COUNT(o.name) one_time_obj_count
		FROM zogo_production.objective_claim oc
		LEFT JOIN user u ON u.id = oc.user_id
		LEFT JOIN objective o ON o.id = oc.objective_id
		WHERE u.institution_id = 20048
			AND o.time_period = 'one_time'
		GROUP BY user_id
		) one_time_user
		WHERE one_time_obj_count >= 1
	) AS modules_per_user
'''

query(mod_per_user_one_obj_query)

,modules_per_user
0,95.9956


In [10]:
##(Number of modules completed)/(user) for users who claimed at least two one-time objectives

mod_per_user_two_obj_query = '''
SELECT
    (SELECT count(*)
    FROM education_completed_module ecm
    JOIN user u on u.id = ecm.user_id
    WHERE u.institution_id = 20048
        AND ecm.date_completed >= '20220903'
        AND u.id IN 
			(
            SELECT user_id
				FROM
				(SELECT
					user_id,
					COUNT(o.name) one_time_obj_count
				FROM zogo_production.objective_claim oc
				LEFT JOIN user u ON u.id = oc.user_id
				LEFT JOIN objective o ON o.id = oc.objective_id
				WHERE u.institution_id = 20048
					AND o.time_period = 'one_time'
				GROUP BY user_id
				) one_time_user
				WHERE one_time_obj_count >= 2
            )
	)
/
    (SELECT count(user_id)
		FROM
		(SELECT
			user_id,
			COUNT(o.name) one_time_obj_count
		FROM zogo_production.objective_claim oc
		LEFT JOIN user u ON u.id = oc.user_id
		LEFT JOIN objective o ON o.id = oc.objective_id
		WHERE u.institution_id = 20048
			AND o.time_period = 'one_time'
		GROUP BY user_id
		) one_time_user
		WHERE one_time_obj_count >= 2
	) AS modules_per_user
'''

query(mod_per_user_two_obj_query)

,modules_per_user
0,112.0103


In [11]:
##(Number of modules completed)/(user) for users who claimed all three one-time objectives

mod_per_user_three_obj_query = '''
SELECT
    (SELECT count(*)
    FROM education_completed_module ecm
    JOIN user u on u.id = ecm.user_id
    WHERE u.institution_id = 20048
        AND ecm.date_completed >= '20220903'
        AND u.id IN 
			(
            SELECT user_id
				FROM
				(SELECT
					user_id,
					COUNT(o.name) one_time_obj_count
				FROM zogo_production.objective_claim oc
				LEFT JOIN user u ON u.id = oc.user_id
				LEFT JOIN objective o ON o.id = oc.objective_id
				WHERE u.institution_id = 20048
					AND o.time_period = 'one_time'
				GROUP BY user_id
				) one_time_user
				WHERE one_time_obj_count >= 3
            )
	)
/
    (SELECT count(user_id)
		FROM
		(SELECT
			user_id,
			COUNT(o.name) one_time_obj_count
		FROM zogo_production.objective_claim oc
		LEFT JOIN user u ON u.id = oc.user_id
		LEFT JOIN objective o ON o.id = oc.objective_id
		WHERE u.institution_id = 20048
			AND o.time_period = 'one_time'
		GROUP BY user_id
		) one_time_user
		WHERE one_time_obj_count >= 3
	) AS modules_per_user
'''

query(mod_per_user_three_obj_query)

,modules_per_user
0,151.3333


#### Objectives claimed/user based on cohort

In [12]:
##(Number of objectives claimed)/(user) for all users who logged in after rollout of one-time objectives

obj_per_user_all_query = '''
SELECT
    (SELECT count(*)
    FROM objective_claim oc
    JOIN user u on u.id = oc.user_id
    WHERE u.institution_id = 20048
        AND oc.date_claimed >= '20220903'
        AND u.id IN 
			(
            SELECT 
				DISTINCT(lt.user_id)
			FROM zogo_production.login_tracker lt
			LEFT JOIN user u ON u.id = lt.user_id
			WHERE u.institution_id = 20048
			AND lt.date_created >= '20220903'
			GROUP BY user_id
            )
	)
/
    (SELECT COUNT(*)
    FROM (SELECT 
				DISTINCT(lt.user_id),
				min(lt.date_created) AS date_login
			FROM zogo_production.login_tracker lt
			LEFT JOIN user u ON u.id = lt.user_id
			WHERE u.institution_id = 20048
			AND lt.date_created >= '20220903'
			GROUP BY user_id) all_users
	) AS obj_per_user
'''

query(obj_per_user_all_query)

,obj_per_user
0,2.0514


In [13]:
##(Number of objectives claimed)/(user) for users who claimed at least one one-time objective 

obj_per_user_one_obj_query = '''
SELECT
    (SELECT count(*)
    FROM objective_claim oc
    JOIN user u on u.id = oc.user_id
    WHERE u.institution_id = 20048
        AND oc.date_claimed >= '20220903'
        AND u.id IN 
			(
            SELECT user_id
				FROM
				(SELECT
					user_id,
					COUNT(o.name) one_time_obj_count
				FROM zogo_production.objective_claim oc
				LEFT JOIN user u ON u.id = oc.user_id
				LEFT JOIN objective o ON o.id = oc.objective_id
				WHERE u.institution_id = 20048
					AND o.time_period = 'one_time'
				GROUP BY user_id
				) one_time_user
				WHERE one_time_obj_count >= 1
            )
	)
/
    (SELECT count(user_id)
		FROM
		(SELECT
			user_id,
			COUNT(o.name) one_time_obj_count
		FROM zogo_production.objective_claim oc
		LEFT JOIN user u ON u.id = oc.user_id
		LEFT JOIN objective o ON o.id = oc.objective_id
		WHERE u.institution_id = 20048
			AND o.time_period = 'one_time'
		GROUP BY user_id
		) one_time_user
		WHERE one_time_obj_count >= 1
	) AS obj_per_user
'''

query(obj_per_user_one_obj_query)

,obj_per_user
0,34.0568


In [14]:
##(Number of objectives claimed)/(user) for users who claimed at least two one-time objectives 

obj_per_user_two_obj_query = '''
SELECT
    (SELECT count(*)
    FROM objective_claim oc
    JOIN user u on u.id = oc.user_id
    WHERE u.institution_id = 20048
        AND oc.date_claimed >= '20220903'
        AND u.id IN 
			(
            SELECT user_id
				FROM
				(SELECT
					user_id,
					COUNT(o.name) one_time_obj_count
				FROM zogo_production.objective_claim oc
				LEFT JOIN user u ON u.id = oc.user_id
				LEFT JOIN objective o ON o.id = oc.objective_id
				WHERE u.institution_id = 20048
					AND o.time_period = 'one_time'
				GROUP BY user_id
				) one_time_user
				WHERE one_time_obj_count >= 2
            )
	)
/
    (SELECT count(user_id)
		FROM
		(SELECT
			user_id,
			COUNT(o.name) one_time_obj_count
		FROM zogo_production.objective_claim oc
		LEFT JOIN user u ON u.id = oc.user_id
		LEFT JOIN objective o ON o.id = oc.objective_id
		WHERE u.institution_id = 20048
			AND o.time_period = 'one_time'
		GROUP BY user_id
		) one_time_user
		WHERE one_time_obj_count >= 2
	) AS obj_per_user
'''

query(obj_per_user_two_obj_query)

,obj_per_user
0,39.3144


In [15]:
##(Number of objectives claimed)/(user) for users who claimed all three one-time objectives 

obj_per_user_three_obj_query = '''
SELECT
    (SELECT count(*)
    FROM objective_claim oc
    JOIN user u on u.id = oc.user_id
    WHERE u.institution_id = 20048
        AND oc.date_claimed >= '20220903'
        AND u.id IN 
			(
            SELECT user_id
				FROM
				(SELECT
					user_id,
					COUNT(o.name) one_time_obj_count
				FROM zogo_production.objective_claim oc
				LEFT JOIN user u ON u.id = oc.user_id
				LEFT JOIN objective o ON o.id = oc.objective_id
				WHERE u.institution_id = 20048
					AND o.time_period = 'one_time'
				GROUP BY user_id
				) one_time_user
				WHERE one_time_obj_count >= 3
            )
	)
/
    (SELECT count(user_id)
		FROM
		(SELECT
			user_id,
			COUNT(o.name) one_time_obj_count
		FROM zogo_production.objective_claim oc
		LEFT JOIN user u ON u.id = oc.user_id
		LEFT JOIN objective o ON o.id = oc.objective_id
		WHERE u.institution_id = 20048
			AND o.time_period = 'one_time'
		GROUP BY user_id
		) one_time_user
		WHERE one_time_obj_count >= 3
	) AS obj_per_user
'''

query(obj_per_user_three_obj_query)

,obj_per_user
0,51.8841


#### 10D Retention Rate

In [16]:
#10 day retention rate for all users who logged in after rollout of one-time objectives


retention_10D_all_query = '''
SELECT
    (SELECT COUNT(DISTINCT(user_id))
	FROM login_tracker lt
	WHERE lt.user_id IN
		(SELECT 
				DISTINCT(lt.user_id)
			FROM zogo_production.login_tracker lt
			LEFT JOIN user u ON u.id = lt.user_id
			WHERE u.institution_id = 20048
			AND lt.date_created >= '20220903'
			GROUP BY user_id
					)
	AND lt.date_created >= '20220912'
	)
/
    (SELECT COUNT(DISTINCT(user_id))
	FROM login_tracker lt
	WHERE lt.user_id IN
		(SELECT 
				DISTINCT(lt.user_id)
			FROM zogo_production.login_tracker lt
			LEFT JOIN user u ON u.id = lt.user_id
			WHERE u.institution_id = 20048
			AND lt.date_created >= '20220903'
			GROUP BY user_id
					)
	AND lt.date_created >= '20220903'
	) AS retention_10D
'''

query(retention_10D_all_query)

,retention_10D
0,0.8189


In [17]:
#10 day retention rate for users who completed at least one objective

retention_10D_one_query = '''
SELECT
    (SELECT COUNT(DISTINCT(user_id))
	FROM login_tracker lt
	WHERE lt.user_id IN
		(SELECT user_id
					FROM
					(SELECT
						user_id,
						COUNT(o.name) one_time_obj_count
					FROM zogo_production.objective_claim oc
					LEFT JOIN user u ON u.id = oc.user_id
					LEFT JOIN objective o ON o.id = oc.objective_id
					WHERE u.institution_id = 20048
						AND o.time_period = 'one_time'
					GROUP BY user_id
					) one_time_user
					WHERE one_time_obj_count >= 1
					)
	AND lt.date_created >= '20220912'
	)
/
    (SELECT COUNT(DISTINCT(user_id))
	FROM login_tracker lt
	WHERE lt.user_id IN
		(SELECT user_id
					FROM
					(SELECT
						user_id,
						COUNT(o.name) one_time_obj_count
					FROM zogo_production.objective_claim oc
					LEFT JOIN user u ON u.id = oc.user_id
					LEFT JOIN objective o ON o.id = oc.objective_id
					WHERE u.institution_id = 20048
						AND o.time_period = 'one_time'
					GROUP BY user_id
					) one_time_user
					WHERE one_time_obj_count >= 1
					)
	AND lt.date_created >= '20220903'
	) AS retention_10D
'''

query(retention_10D_one_query)

,retention_10D
0,0.8908


In [18]:
#10 day retention rate for users who completed at least two objectives

retention_10D_two_query = '''
SELECT
    (SELECT COUNT(DISTINCT(user_id))
	FROM login_tracker lt
	WHERE lt.user_id IN
		(SELECT user_id
					FROM
					(SELECT
						user_id,
						COUNT(o.name) one_time_obj_count
					FROM zogo_production.objective_claim oc
					LEFT JOIN user u ON u.id = oc.user_id
					LEFT JOIN objective o ON o.id = oc.objective_id
					WHERE u.institution_id = 20048
						AND o.time_period = 'one_time'
					GROUP BY user_id
					) one_time_user
					WHERE one_time_obj_count >= 2
					)
	AND lt.date_created >= '20220912'
	)
/
    (SELECT COUNT(DISTINCT(user_id))
	FROM login_tracker lt
	WHERE lt.user_id IN
		(SELECT user_id
					FROM
					(SELECT
						user_id,
						COUNT(o.name) one_time_obj_count
					FROM zogo_production.objective_claim oc
					LEFT JOIN user u ON u.id = oc.user_id
					LEFT JOIN objective o ON o.id = oc.objective_id
					WHERE u.institution_id = 20048
						AND o.time_period = 'one_time'
					GROUP BY user_id
					) one_time_user
					WHERE one_time_obj_count >= 2
					)
	AND lt.date_created >= '20220903'
	) AS retention_10D
'''

query(retention_10D_two_query)

,retention_10D
0,0.9175


In [19]:
#10 day retention rate for users who completed all three objectives

retention_10D_three_query = '''
SELECT
    (SELECT COUNT(DISTINCT(user_id))
	FROM login_tracker lt
	WHERE lt.user_id IN
		(SELECT user_id
					FROM
					(SELECT
						user_id,
						COUNT(o.name) one_time_obj_count
					FROM zogo_production.objective_claim oc
					LEFT JOIN user u ON u.id = oc.user_id
					LEFT JOIN objective o ON o.id = oc.objective_id
					WHERE u.institution_id = 20048
						AND o.time_period = 'one_time'
					GROUP BY user_id
					) one_time_user
					WHERE one_time_obj_count >= 3
					)
	AND lt.date_created >= '20220912'
	)
/
    (SELECT COUNT(DISTINCT(user_id))
	FROM login_tracker lt
	WHERE lt.user_id IN
		(SELECT user_id
					FROM
					(SELECT
						user_id,
						COUNT(o.name) one_time_obj_count
					FROM zogo_production.objective_claim oc
					LEFT JOIN user u ON u.id = oc.user_id
					LEFT JOIN objective o ON o.id = oc.objective_id
					WHERE u.institution_id = 20048
						AND o.time_period = 'one_time'
					GROUP BY user_id
					) one_time_user
					WHERE one_time_obj_count >= 3
					)
	AND lt.date_created >= '20220903'
	) AS retention_10D
'''

query(retention_10D_three_query)

,retention_10D
0,0.9565


## Visions FCU

In [3]:
#All one-time objectives completed:
query('''SELECT oc.user_id, oc.date_claimed, o.name, o.time_period
FROM objective_claim oc
LEFT JOIN user u ON u.id = oc.user_id
LEFT JOIN objective o ON o.id = oc.objective_id
WHERE u.institution_id = 7401
AND o.time_period = 'one_time'
''')

,user_id,date_claimed,name,time_period
0,860020,2022-07-12 16:54:12,Financial Wellness,one_time
1,1234156,2022-07-12 19:48:07,Financial Wellness,one_time
2,1433525,2022-07-12 19:58:23,Financial Wellness,one_time
3,1118382,2022-07-12 20:57:44,Financial Wellness,one_time
4,1272279,2022-07-13 00:28:39,Financial Wellness,one_time
...,...,...,...,...
277,1178784,2022-10-21 18:26:09,Complete 5 Modules,one_time
278,934992,2022-10-21 18:50:17,Complete 5 Modules,one_time
279,2190702,2022-10-22 00:10:51,Complete 5 Modules,one_time
280,1661132,2022-10-22 02:45:18,Complete 5 Modules,one_time


In [10]:
#Modules/user for all Visions FCU users who completed 5 or more modules:

query('''
SELECT
(
SELECT count(*)
FROM education_completed_module ecm
WHERE ecm.user_id IN
(SELECT user_id
FROM
(SELECT ecm.user_id, count(education_module_id) AS mod_count
FROM education_completed_module ecm
LEFT JOIN user u ON u.id = ecm.user_id
WHERE u.institution_id = 7401
GROUP BY user_id) visions_users
WHERE mod_count >= 5)
)
/
(
SELECT count(user_id)
FROM
(SELECT ecm.user_id, count(education_module_id) AS mod_count
FROM education_completed_module ecm
LEFT JOIN user u ON u.id = ecm.user_id
WHERE u.institution_id = 7401
GROUP BY user_id) visions_users
WHERE mod_count >= 5) AS modules_per_user
''')

,modules_per_user
0,47.6295


In [14]:
#Modules/user for all Visions FCU users who claimed the Complete 5 Modules objective:

query('''
SELECT
(
SELECT count(education_module_id)
FROM education_completed_module ecm
WHERE ecm.user_id IN 
(SELECT oc.user_id
FROM objective_claim oc
LEFT JOIN user u ON u.id = oc.user_id
LEFT JOIN objective o ON o.id = oc.objective_id
WHERE u.institution_id = 7401
AND o.time_period = 'one_time'
AND o.name = 'Complete 5 Modules'
)
)/
(
SELECT count(user_id)
FROM 
(SELECT oc.user_id
FROM objective_claim oc
LEFT JOIN user u ON u.id = oc.user_id
LEFT JOIN objective o ON o.id = oc.objective_id
WHERE u.institution_id = 7401
AND o.time_period = 'one_time'
AND o.name = 'Complete 5 Modules'
) one_time_users
) AS modules_per_user
''')

,modules_per_user
0,200.9432


In [12]:
#10D retention rate for all Visions FCU users who claimed the Complete 5 Modules objective:

query('''
SELECT
    (SELECT COUNT(DISTINCT(user_id))
	FROM login_tracker lt
	WHERE lt.user_id IN
		(SELECT user_id
		FROM
		(SELECT ecm.user_id, count(education_module_id) AS mod_count
		FROM education_completed_module ecm
		LEFT JOIN user u ON u.id = ecm.user_id
		WHERE u.institution_id = 7401
		GROUP BY user_id) visions_users
		WHERE mod_count >= 5
		)
	AND lt.date_created >= '20221001'
	)
/
    (SELECT COUNT(DISTINCT(user_id))
	FROM login_tracker lt
	WHERE lt.user_id IN
		(SELECT user_id
		FROM
		(SELECT ecm.user_id, count(education_module_id) AS mod_count
		FROM education_completed_module ecm
		LEFT JOIN user u ON u.id = ecm.user_id
		WHERE u.institution_id = 7401
		GROUP BY user_id) visions_users
		WHERE mod_count >= 5
		)
	AND lt.date_created >= '20220922'
	) AS retention_10D
''')

,retention_10D
0,0.7778


In [11]:
#10D retention rate for all Visions FCU users who claimed the Complete 5 Modules objective:
query('''
SELECT
    (SELECT COUNT(DISTINCT(user_id))
	FROM login_tracker lt
	WHERE lt.user_id IN
		(SELECT oc.user_id
		FROM objective_claim oc
		LEFT JOIN user u ON u.id = oc.user_id
		LEFT JOIN objective o ON o.id = oc.objective_id
		WHERE u.institution_id = 7401
		AND o.time_period = 'one_time'
		AND o.name = 'Complete 5 Modules'
		)
	AND lt.date_created >= '20221001'
	)
/
    (SELECT COUNT(DISTINCT(user_id))
	FROM login_tracker lt
	WHERE lt.user_id IN
		(SELECT oc.user_id
		FROM objective_claim oc
		LEFT JOIN user u ON u.id = oc.user_id
		LEFT JOIN objective o ON o.id = oc.objective_id
		WHERE u.institution_id = 7401
		AND o.time_period = 'one_time'
		AND o.name = 'Complete 5 Modules'
					)
	AND lt.date_created >= '20220922'
	) AS retention_10D
''')

,retention_10D
0,0.9091


## RRCU Users

In [13]:
#Modules/user for all Visions FCU users who completed 5 or more modules:

query('''
SELECT
(
SELECT count(*)
FROM education_completed_module ecm
WHERE ecm.user_id IN
(SELECT user_id
FROM
(SELECT ecm.user_id, count(education_module_id) AS mod_count
FROM education_completed_module ecm
LEFT JOIN user u ON u.id = ecm.user_id
WHERE u.institution_id = 19634
GROUP BY user_id) rrcu_users
WHERE mod_count >= 5)
)
/
(
SELECT count(user_id)
FROM
(SELECT ecm.user_id, count(education_module_id) AS mod_count
FROM education_completed_module ecm
LEFT JOIN user u ON u.id = ecm.user_id
WHERE u.institution_id = 19634
GROUP BY user_id) rrcu_users
WHERE mod_count >= 5) AS modules_per_user
''')

,modules_per_user
0,29.8653


In [15]:
#Modules/user for all Visions FCU users who claimed the Complete 5 Modules objective:

query('''
SELECT
(
SELECT count(education_module_id)
FROM education_completed_module ecm
WHERE ecm.user_id IN 
(SELECT oc.user_id
FROM objective_claim oc
LEFT JOIN user u ON u.id = oc.user_id
LEFT JOIN objective o ON o.id = oc.objective_id
WHERE u.institution_id = 19634
AND o.time_period = 'one_time'
AND o.name = 'Complete 5 Modules'
)
)/
(
SELECT count(user_id)
FROM 
(SELECT oc.user_id
FROM objective_claim oc
LEFT JOIN user u ON u.id = oc.user_id
LEFT JOIN objective o ON o.id = oc.objective_id
WHERE u.institution_id = 19634
AND o.time_period = 'one_time'
AND o.name = 'Complete 5 Modules'
) one_time_users
) AS modules_per_user
''')

,modules_per_user
0,140.6111
